In [4]:
import bench_utils
import shapely
from shapely.geometry import shape
from tqdm import tqdm
from intersection.plotting import *
import matplotlib.pyplot as plt
import json
import geopandas as gpd
import pickle

MAX_INTERSECTIONS = 100000
MAX_ONE_SHAPE = 5

intersections = []
nbr_intersects = 0
nbr_false = 0
vertices = [[], []]

def append_data_set(file_name, intersections, PRINT_AND_DRAW=True):
    global nbr_intersects
    global nbr_false
    #df, _ = bench_utils.read_dataset(file_name)
    if file_name.endswith(".shp"):
        df = gpd.read_file(file_name)
    else:
        df, _ = bench_utils.load_shp_files(file_name)

    df =  df.sample(frac=1)
    # Add an index row
    df = df.reset_index()
    shp_cnt = len(df)

    with tqdm(total=MAX_INTERSECTIONS) as pbar:
        for i in tqdm(range(shp_cnt), disable=True):
            pbar.set_postfix(iteration=len(intersections))
            if len(intersections) >= MAX_INTERSECTIONS: 
                break
            #shp_1 = shape(df.iloc[i])
            shp_1 = df.iloc[i].geometry
            x_l_1, y_b_1, x_r_1, y_t_1 = shp_1.bounds
            from shapely.geometry import box
            shapes_within_bbox = df[df.geometry.intersects(box(x_l_1, y_b_1, x_r_1, y_t_1))]
            #shapes_within_bbox = df
            added = 0
            for r, shp_2_row in shapes_within_bbox.iterrows():
                shp_2 = shp_2_row.geometry
                if df.iloc[i]['index'] <= shp_2_row['index']:
                    continue
                x_l_2, y_b_2, x_r_2, y_t_2 = shp_2.bounds
                bbox = [max(x_l_1, x_l_2), max(y_b_1, y_b_2), min(x_r_1, x_r_2), min(y_t_1, y_t_2)]
                x_l, y_b, x_r, y_t = bbox
                if x_r < x_l or y_t < y_b:
                    continue

                types = [shp_1.geom_type, shp_2.geom_type]
                types.sort()
                t_1, t_2 = types
                type = t_1 + t_2
                if type == "LineStringLineString" and random.randint(0, 100) != 0:
                    continue
                
                added += 1
                if added >= MAX_ONE_SHAPE:
                    continue
                intersects = shp_1.intersects(shp_2)
                intersections += [[type, shapely.to_wkb(shp_1), shapely.to_wkb(shp_2), str(intersects)]]
                if intersects:
                    nbr_intersects += 1
                    vertices[0].append(shapely.get_num_coordinates(shp_1))
                    vertices[0].append(shapely.get_num_coordinates(shp_2))
                else:
                    nbr_false += 1
                    vertices[1].append(shapely.get_num_coordinates(shp_1))
                    vertices[1].append(shapely.get_num_coordinates(shp_2))
                if PRINT_AND_DRAW:
                    print(type, intersects)
                    plot_geometry(shp_1)
                    plot_geometry(shp_2)
                    plt.show()


#append_data_set("QGIS/Intersection_Tricky_Cases.shp", intersections, PRINT_AND_DRAW=False)
#append_data_set("data/lund_building_highway.json", intersections, False)
#append_data_set("data/world.json", intersections, False)
#append_data_set("data/china-latest-free/gis_osm_water_a_free_1.shp", intersections, False)
#append_data_set("data/sweden-latest-free", intersections, False)
#append_data_set("QGIS/Report_Visualizations.shp", intersections, PRINT_AND_DRAW=False)
append_data_set("data/final_country_borders.shp", intersections, PRINT_AND_DRAW=False)
#print(f"Total number intersects: {nbr_intersects}, false: {nbr_false}")
#print(f"Avg vertex count true: {np.average(vertices[0])}, false: {np.average(vertices[1])}")
random.shuffle(intersections)

# f = open("data/intersection/latest_export.json", "w")
# f.write(json.dumps(intersections))
# f.close()
with open("data/intersection/latest_export.pkl", , 'wb') as f:
    pickle.dump(intersections, f)

  0%|          | 0/100000 [00:26<?, ?it/s, iteration=10781]


FileNotFoundError: [Errno 2] No such file or directory: 'data/intersection/latest_export.pkl'